In [18]:
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg19
from pathlib import Path
import joblib

In [19]:
# Load keras VGG19 model that was pre-trained on against the ImageNet databse
model = vgg19.VGG19()

In [20]:
# Load the image file
# Resize it to 224*224 pixels (required by this model)
img = image.image_utils.load_img("./images/bay.jpg", target_size=(224, 224))

# Convert the image to numpy array
x = image.image_utils.img_to_array(img)

# Add fourth dimension since keras expects a list of images
x = np.expand_dims(x, axis=0)

In [21]:
# Normalize the input image's pixel values to the range used when training the neural network
x = vgg19.preprocess_input(x)

In [22]:
# Run the image through the deep neural network to perform prediction
predictions = model.predict(x)

1/1 [==============================] - 0s 296ms/step


In [23]:
# Look up the the names of the prediction classes.
# Index Zero is the result for the first image
predicted_classes = vgg19.decode_predictions(predictions)

In [24]:
# Print the top predictions for the image
for imagenet_id, name, likelihood in predicted_classes[0]:
    print(f"Prediction: {name} – {likelihood:.2f}")

Prediction: seashore – 0.60
Prediction: promontory – 0.23
Prediction: sandbar – 0.07
Prediction: lakeside – 0.04
Prediction: breakwater – 0.02


In [30]:
# Import training data
dog_path = Path("./images/training_data/dogs")
not_dog_path = Path("./images/training_data/not_dogs")

In [31]:
# Load images in to the images array and create labels (1 dog, 0 not dog)
images = []
labels = []

# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each dog image, the label should 1
    labels.append(1)

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each not-dog image, the label should 0
    labels.append(0)

In [32]:
# Create a single numpy array with all the images loaded
X_train = np.array(images)

# Create a single numpy array with all the labels created
Y_train = np.array(labels)

# Normalize image data to (0 - 1) range
X_train = vgg19.preprocess_input(X_train)

# Load a pre-trained neural network to use as feature extractor
pre_trained_nn = vgg19.VGG19(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_X = pre_trained_nn.predict(X_train)

2/2 [==============================] - 1s 294ms/step


In [33]:
# Save the the array of extracted features to a file
joblib.dump(features_X, "X_train.dat")

['X_train.dat']

In [34]:
# Save the matching array of expected values to a file
joblib.dump(Y_train, "Y_train.dat")

['Y_train.dat']